# Doge


Train the `Doge` small language model proposed in the paper [Wonderful Matrices](https://arxiv.org/abs/2412.11834).
Doge is based on the Transformers framework, replacing the `Multi-Head Attention` in the sequence transformation part with `Dynamic Mask Attention`, and replacing the `MLP` in the state transformation part with `CDMoE`.

![doge_architecture](../assets/doge_architecture.png)

## PreTrain

### Download Pre-Training and Fine-Tuning Datasets


For the pre-training dataset, we selected the high-quality text `fineweb-edu-dedup`, the synthetic instruction dataset `cosmopedia-v2`, and supplemented it with `python-edu` and `open-web-math` to ensure the model's code and mathematical capabilities.


> Note: Due to the large size of the dataset, at least 2TB of storage space is required.

In [ ]:
# Padding save path, cache path and number of processes
!python ./examples/pretrain/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### Preprocess Datasets


We need to use the `tokenizer` to convert the dataset into `input_ids` and `attention_mask` that the model can accept.
Doge uses the `LlamaTokenizer`, which has a vocabulary size of `32768`, and uses the `[INST]` and `[/INST]` tags to mark instructions. It also includes utility tokens, but we won't use them here.
Datasets like cosmopedia-v2 include two fields, `prompt` and `text`, which we will mark as user content and assistant content.

```python
conversation = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
return tokenizer.apply_chat_template(conversation, tokenize=True, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_dict=True)
```


Of course, you can also add some instruction prompts yourself.


```python
conversation = [
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I am an AI assistant named `Doge`, I am a language model trained by `Shi Jingze` based on the `Doge` architecture, and my task is to provide appropriate answers and support to users based on their questions and requests."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
```

In [ ]:
# Padding dataset path, save path, tokenizer path, number of samples, max length and number of processes
# NOTE: We only keep 160B tokens dataset, the ratio of fineweb-edu:cosmopedia-v2:python-edu:open-web-math = 7:2:0.5:0.5, if you need to train larger model, please increase the scale of the dataset by yourself
!python ./examples/pretrain/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --train_examples 81920000 --test_examples 1000 --max_length 2048 --num_proc 16

### Concatenate Datasets


We combine the fineweb-edu_tokenized, cosmopedia-v2, python-edu, and open-web-math datasets into the `pretrain` dataset.
Then shuffle the order `seed=233`, and split out `1,000` samples as the test set.

In [ ]:
# Padding dataset path, save path, number of samples and number of processes
!python ./examples/pretrain/scripts/concatenate_datasets.py --datasets_dir ./datasets --save_dir ./datasets --train_examples 81920000 --test_examples 1000 --num_proc 16

### Configure Model


We configure a `20M` small model for training and testing.

| Model | Params | n_layers | d_model | d_ff | n_heads | n_exprets | n_expert_heads | n_expert_pre_head |
|---|---|---|---|---|---|---|---|---|
| Doge-20M | 20M | 4 | 256 | 1024 | 2 | - | - | - |
| Doge-MoE-20M | 20M | 4 | 256 | 1024 | 2 | 1024 | 2 | 4 |
| Doge-60M | 60M | 8 | 512 | 2048 | 4 | - | - | - |
| Doge-MoE-80M | 80M | 8 | 512 | 2048 | 4 | 2048 | 4 | 8 |
| Doge-160M | 160M | 12 | 768 | 3072 | 6 | - | - | - |
| Doge-MoE-230M | 230M | 12 | 768 | 3072 | 6 | 3072 | 6 | 12 |
| Doge-320M | 320M | 16 | 1024 | 4096 | 8 | - | - | - |
| Doge-MoE-500M | 500M | 16 | 1024 | 4096 | 8 | 4096 | 8 | 16 |


- n_layers is the number of decoder layers in the model
- d_model is the hidden layer dimension of the model
- n_heads is the number of heads of multi-head attention, d_model // n_heads is best kept above 64


> The `Doge-MoE` model can inherit the dense activation parameters of the `Doge` model, and increase the sparse activation parameters by setting `n_experts`, `n_expert_heads`, `n_expert_pre_head`.

### Configure Pre-Training Hyperparameters

| Params | tokens | num_train_epochs | per_epoch_max_steps | accumulate_steps | learning_rate | warmup_ratio | weight_decay | min_lr_rate |
|--------|--------|------------------|---------------------|------------------|---------------|--------------|--------------|-------------|
| 20M    | 5B     | 2                | 10,000              | 128              | 8e-4          | 0.1          | 0.01         | 0.1         |
| 60M    | 20B    | 2                | 20,000              | 256              | 6e-4          | 0.1          | 0.01         | 0.1         |
| 160M   | 80B    | 2                | 40,000              | 512              | 5e-4          | 0.1          | 0.01         | 0.1         |
| 320M   | 320B   | 2                | 80,000              | 1024             | 4e-4          | 0.1          | 0.01         | 0.1         |

### Pre-Training Model

In [ ]:
# Padding config path, logging path, output path and tokenizer path, if needed, you can also specify checkpoint path to resume training --resume_from_checkpoint
!python ./examples/pretrain/scripts/pretrain.py --config_path ./examples/pretrain/configs/Doge-20M.yaml --logging_dir ./logs --output_dir ./results --tokenizer_path ./examples/tokenizer --resume_from_checkpoint <path_to_checkpoint>

### Usage


After training is complete, we can use `AutoModelForCausalLM` of `Transformers` to load the model, and use `AutoTokenizer` to load `LlamaTokenizer`.

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("JingzeShi/Doge-20M")
model = AutoModelForCausalLM.from_pretrained("JingzeShi/Doge-20M", trust_remote_code=True)

In [ ]:
inputs = tokenizer("Hey how are you doing?", return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.batch_decode(out))

## FineTune

### Download Fine-Tuning Datasets


For the fine-tuning dataset, we selected the synthetic instruction dataset `smoltalk` for supervised fine-tuning.

In [ ]:
# Padding save path, cache path and number of processes
!python ./examples/finetune/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### Process Fine-Tuning Datasets


We'll apply Fine-Tuning datasets with `chat templete` .

In [ ]:
# Padding dataset path, save path, tokenizer path, number process.
!python ./examples/finetune/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --num_proc 8

### Fine-Tuning Model

In [ ]:
# Padding pretrained model name or path, config path, logging path, output path, if needed, you can also specify checkpoint path to resume training --resume_from_checkpoint
!python ./examples/finetune/scripts/sft.py --pretrained_model_name_or_path JingzeShi/Doge-20M --config_path ./examples/finetune/configs/Doge-20M-Instruct.yaml --logging_dir ./logs --output_dir ./results --resume_from_checkpoint <path_to_checkpoint>

### Usage

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextStreamer

tokenizer = AutoTokenizer.from_pretrained("JingzeShi/Doge-20M-Instruct")
model = AutoModelForCausalLM.from_pretrained("JingzeShi/Doge-20M-Instruct", trust_remote_code=True)

In [12]:
generation_config = GenerationConfig(
      max_new_tokens=100, 
      use_cache=True, 
      do_sample=True, 
      temperature=0.8, 
      repetition_penalty=1.0
)
steamer = TextStreamer(
      tokenizer=tokenizer, 
      skip_prompt=True
)

In [ ]:
prompt = "Hi, how are you doing today?"

conversation = [
      {"role": "user", "content": prompt}
]
inputs = tokenizer.apply_chat_template(
    conversation=conversation,
    tokenize=True,
    return_tensors="pt",
)

outputs = model.generate(
    inputs, 
    tokenizer=tokenizer,
    generation_config=generation_config, 
    streamer=steamer
)

## Evaluation


First, install `miniconda`.


```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
```


Then create an evaluation environment.


```bash
conda create -n lighteval python=3.10.12 
conda activate lighteval
pip install lighteval[accelerate]
```


Finally, we run the evaluation script.


if you use Linux, you can run the following command.


```bash
bash ./examples/evaluate/eval_downstream_tasks.sh
```


if you use Windows, you can run the following command.


```bash
. ./examples/evaluate/eval_downstream_tasks.ps1
```


> NOTE: The MODEL in the script can also be filled with the saved checkpoint path, just need to register the save to run.